# Dataset
* Site Viva real imóveis
* Local escolhido para busca de imóveis: Florianópolis, Santa Catarina, Brasil
* Período analisado: janeiro 2021 
* Cabe ressaltar que é período de verão, tendo preços inflados, tornando os dados enviesados

## Variáveis coletadas
* Produto: tipos de imóveis
* Área: metragem quadrada do imóvel
* Quartos: número total de quartos no imóvel
* Banheiros: número total de banheiro no imóvel
* Garagem: número total de vagas de garagem do imóvel
* Adress: endereço, bairro e cidade em que o imóvel se encontra
* Preço: valor em reais para locação cobrado mensal ou diariamente

## Preparando o ambiente

In [ ]:
# importações
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [ ]:
# Definindo estrutura do dataframe
df_orig = pd.DataFrame(columns=['product', 
                               'area', 
                               'room', 
                               'bath', 
                               'garage', 
                               'adress', 
                               'price'])

## Tentativa de realizar com beatiful soup
- Alteração diretamente pela URL -> sem sucesso

In [ ]:
page = 1

while page <=3:
    # Viva Real homepage
    url = f'https://www.vivareal.com.br/aluguel/santa-catarina/florianopolis/?pagina={page}'
    sleep(3)    
    # Usaremos o request para realizar a requisição
    vr_response = requests.get(url)
    sleep(2)
    # Conversão da resposta para HTML Usando Beautiful Soup
    vr_soup = BeautifulSoup(vr_response.text, 'html.parser')
    sleep(2)
    prod_lists = [product.text.lower() for product in vr_soup.find_all('span', {'class':'property-card__title'})]
    area_lists = [area.text.lower() for area in vr_soup.find_all('li', {'class':'property-card__detail-area'})]
    room_lists = [room.text.lower() for room in vr_soup.find_all('li', {'class':'property-card__detail-room'})]
    bath_lists = [bath.text.lower() for bath in vr_soup.find_all('li', {'class': 'property-card__detail-bathroom'})]
    garage_lists = [garage.text.lower() for garage in vr_soup.find_all('li', {'class':'property-card__detail-garage'})]
    adress_lists = [adress.text.lower() for adress in vr_soup.find_all('span', {'class':'property-card__address'})]
    price_lists = [price.text.lower() for price in vr_soup.find_all('div', {'class':'property-card__price'})]
    page += 1
    df_orig = df_orig.append(pd.DataFrame({'product':prod_lists, 
                                           'area':area_lists, 
                                           'room':room_lists, 
                                           'bath':bath_lists, 
                                           'garage':garage_lists, 
                                           'adress':adress_lists, 
                                           'price':price_lists}))
    print(page,url)

## Acessando o site

In [ ]:
# Preparado para acessar
driver = webdriver.Chrome(ChromeDriverManager().install())
sleep(3)

In [ ]:
url = "https://www.vivareal.com.br/aluguel/santa-catarina/florianopolis/"
driver.get(url)
sleep(2)

## Tentativas de realizar com o selenium
- Utilizar comando de click para troca de página -> sem sucesso

In [ ]:
while len(df_orig) < 6800:
    # Gerando os web elements
    prod_lists = driver.find_elements_by_class_name('property-card__title')
    area_lists = driver.find_elements_by_class_name('property-card__detail-area')
    room_lists = driver.find_elements_by_class_name('property-card__detail-room')
    bath_lists = driver.find_elements_by_class_name('property-card__detail-bathroom')
    garage_lists = driver.find_elements_by_class_name('property-card__detail-garage')
    adress_lists = driver.find_elements_by_class_name('property-card__address')
    price_lists = driver.find_elements_by_class_name('property-card__price')
    
    # Gerando as listas de elementos textuais
    prod = [n.text for n in prod_lists][:36]
    area = [n.text for n in area_lists][:36]
    room = [n.text for n in room_lists][:36]
    bath = [n.text for n in bath_lists][:36]
    garage = [n.text for n in garage_lists][:36]
    adress = [n.text for n in adress_lists][:36]
    price = [n.text for n in price_lists][:36]
    
    # Adicionando elementos ao DF
    df_orig = df_orig.append(pd.DataFrame({'product':prod_lists, 
                                           'area':area_lists, 
                                           'room':room_lists, 
                                           'bath':bath_lists, 
                                           'garage':garage_lists, 
                                           'adress':adress_lists, 
                                           'price':price_lists}))
    # Trocando paginação
    bottom = driver.find_element_by_partial_link_text('#pagina=')
    bottom.click()

## Verificando dados & exportando o dataframe

In [ ]:
len(df_orig)

In [ ]:
# Exportando o df para máquina local
df_orig.to_csv('df_org.csv', sep=';')